In [1]:
import time
import glob
import sys
import os
import uuid
import numpy as np
from PIL import Image, ImageDraw
import cv2
import pandas as pd
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [2]:
nb_dir = '/home/naresh/Tarento/primalaynet/layout-model-training/tools/table/'
#sys.path.append(nb_dir)
#sys.path.append(os.path.split(nb_dir)[0])

#import config
from src.services.get_table_cells import mask_tables 

import src.utilities.app_context as app_context
app_context.init()
app_context.application_context = dict({'task_id': 'BM-DUMMY'})





In [3]:
def draw_region(image_path,coords,save_dir,color=(255,125,0), thickness=3):
    
    image = cv2.imread(image_path)
    for region in coords :
            ground = region['boundingBox']['vertices']
            pts = []
            for pt in ground:
                pts.append([int(pt['x']) ,int(pt['y'])])
            cv2.polylines(image, [np.array(pts)],True, color, thickness)
#             cv2.putText(image, str(region['class']), (pts[0][0],pts[0][1]), font,  
#            2, (0,125,255), 3, cv2.LINE_AA)

    image_path = os.path.join(save_dir ,  '{}.png'.format(uuid.uuid4()))            
    cv2.imwrite(image_path , image)


In [4]:
df = pd.DataFrame(columns = ['ID', 'Label']);
ids =[]
label =[]
index =0
image_dir = '/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_tagging/crop_table/'

image = glob.glob(image_dir + '/*.jpg')
for page_path in image:
    #path_to_text_file = save_dir+ page_path.split('/')[-1].split('.jpg')[0]+'.txt'
    #print(page_path)
#     if 'prima_double_column' not in page_path:
#         print(page_path)
    masked_image, reg  = mask_tables(page_path, check=False)
    #cv2.imwrite(image_path , image)
    #text_file = open(path_to_text_file, "w+")
    reg_str =''
    for idx,region in enumerate(reg):
        #cls = region['class'].lower()

        #if cls in ['footer','header','image','others','text','table']:
            #text_file.write(str(idx))
        reg_str = reg_str + str(region['boundingBox']['vertices'][0]['x'])+" "+str(region['boundingBox']['vertices'][0]['y'])+" "+str(region['boundingBox']['vertices'][2]['x'])+" "+str(region['boundingBox']['vertices'][2]['y'])+" "+'table'+' ' 
#                 text_file.write(str(cls_map[cls]))
#                 text_file.write(" "+str(region['boundingBox']['vertices'][0]['x']))
#                 text_file.write(" "+str(region['boundingBox']['vertices'][0]['y']))
#                 text_file.write(" "+str(region['boundingBox']['vertices'][2]['x']))
#                 text_file.write(" "+str(region['boundingBox']['vertices'][2]['y'])+"\n")
        #img = cv2.imread(page_path)
        #img2 = img[region['boundingBox']['vertices'][0]['y']:region['boundingBox']['vertices'][2]['y'],region['boundingBox']['vertices'][0]['x']:region['boundingBox']['vertices'][2]['x']]
        #print(img2)
        #cv2.imwrite(page_path.split('.jpg')[0]+str(idx)+'.jpg',img2)
    
    if len(reg_str)>0:
        print(page_path)
        #ids.append(page_path.split('/')[-1]+str(idx)+'.jpg')
        ids.append(page_path.split('/crop_table/')[-1])
        label.append(reg_str)

        index = index+1


/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_tagging/crop_table/00000925_prima_double_column_0.jpg
/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_tagging/crop_table/00000698_prima_double_column_1.jpg
/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_tagging/crop_table/00000805_prima_double_column_0.jpg
/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_tagging/crop_table/00000799_prima_double_column_0.jpg
/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_tagging/crop_table/00000743_prima_double_column_1.jpg
/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_tagging/crop_table/00000698_prima_double_column_0.jpg
/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_tagging/crop_table/00001122_prima_double_column_0.jpg


In [5]:
df['ID']=ids
df['Label']=label

In [6]:
df.to_csv("/home/naresh/Tarento/primalaynet/layout-model-training/tools/table.csv")

In [7]:
df2 = df.dropna(axis=1)

In [ ]:
df

In [ ]:
save_dir = '/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_extraction_tool/samples'

In [ ]:
image_path = '/home/naresh/Tarento/primalaynet/layout-model-training/tools/Images/LAFEAS_LS-pdf-45_judgement_single_column_.jpg'

In [ ]:
masked_image, line_tables_regions  = mask_tables(image_path, check=False)

In [ ]:
draw_region(image_path,line_tables_regions,save_dir)